## Ноутбук с симуляцией ATL

Импорты:

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
from PIL import Image, ImageDraw
from pyffs import ffsn, iffsn, ffs_shift

### Функция, считающая картину на определенном расстоянии по фотке маски

пока что только для монохроматического пучка (просто саморепродукция)

In [79]:
def compute_atl(mask, wavelength, T_x, T_y, z = None):
    '''
    Функция для вычисления дифракционной картины ATL
    param mask: изображение маски,
    param scale: масштаб маски, [ед./пиксель], 
    param period: период маски, [ед.], 
    param wavelength: длина волны, [ед.]
    param z: расстояние от картины до маски, [1000 * ед.]

    return: изображение картины
    '''

    mask_array = np.array(mask)
    print(mask_array.shape)

    T = [T_x, T_y]
    T_c = [0, 0]
    N_FS = [9, 9]
    F = ffs_shift(ffsn(mask_array, T, T_c, N_FS))
    f_x = np.reshape(np.arange(F.shape[0]) / T_x, (1, -1))
    f_y = np.reshape(np.arange(F.shape[1]) / T_y, (-1, 1))
    # f_y = np.arange(F.shape[1]) / T_y
    if z == None:
        z = 1 * T_x ** 2 / wavelength
    H = np.exp(-1j * np.pi * wavelength * z * (np.power(f_x, 2) + np.power(f_y, 2)))
    result_array = iffsn(F * H, T, T_c, N_FS)
    reps = np.tile(np.array(result_array), (5, 5))
    return Image.fromarray(np.absolute(reps))

Пробуем функцию на примере кружка

In [81]:
mask = Image.open("grayscale_circle.png").convert("L")
period = 1
wavelength = 0.1
z = 7
pic = compute_atl(mask, wavelength, period, period, z)
pic.show()

(256, 256)


### Эту херню сделала ChatGPT

(это не то)

In [31]:
reps = np.tile(np.array(mask), (10, 10)) # Change the number of repetitions as needed

# Perform Fourier transform
fft = np.fft.fft2(reps)

# Calculate Talbot distance
Td = (100 * period**2) / wavelength

# Apply phase shift
phase_shift = np.exp(-1j * np.pi * Td / wavelength * (np.fft.fftfreq(fft.shape[0])[:, np.newaxis]**2 + np.fft.fftfreq(fft.shape[1])**2))
fft_shifted = fft * phase_shift

# Inverse Fourier transform
output = np.fft.ifft2(fft_shifted)
im = Image.fromarray(np.absolute(output))
im.show()